## Detailed article explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541200/custom-loss-functions-in-pytorch-a-comprehensive-guide

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

In [ ]:
# Import necessary libraries
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch import nn, optim


In [82]:
# Load the data

# download the dataset here: https://www.kaggle.com/datasets/shree1992/housedata?select=data.csv

data = pd.read_csv("/content/house_price_dataset.csv")
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [83]:
import pandas as pd

def preprocess_data(df):
    # Step 1: Remove the 'street' column
    df = df.drop(columns=['street'])

    # Step 2: Break the 'date' column into 'year', 'month', and 'day'
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df = df.drop(columns=['date'])

    # Step 3: Perform one-hot encoding on 'city', 'statezip', and 'country' columns using pd.get_dummies
    df = pd.concat([df, pd.get_dummies(df[['city', 'statezip', 'country']], drop_first=True)], axis=1)
    df = df.drop(columns=['city', 'statezip', 'country'])

    return df

data = preprocess_data(data)
len(data.columns)

135

In [85]:
def get_train_test_split(df):
    # Preprocessing steps as before
    # ...

    # Separate the features and the label
    X = df.drop(columns=['price'])  # Features
    y = df['price']                 # Label

    # Split the dataset into a training set and a test set (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_split(data)

In [99]:
# Define the dataset

import torch
from torch.utils.data import Dataset

class HousingDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        data_val = self.X[idx]
        target = self.y[idx]
        return data_val, target


train_dataset = HousingDataset(X_train, y_train)
test_dataset = HousingDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [96]:
# Define the regression model
class RegressionModel(nn.Module):
    def __init__(self, input_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)  # Output layer for regression

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)  # No activation function for the output layer in regression
        return x

input_size = len(X_train.columns)
model = RegressionModel(input_size)

In [93]:
def custom_mse_loss(output, target, threshold=0.20, penalty_weight=2.0):

    # Calculate squared error
    squared_error = (output - target) ** 2

    # Calculate the relative error
    relative_error = torch.abs((output - target) / target)

    # Apply additional weight where the error exceeds the threshold
    weighted_error = torch.where(relative_error > threshold, penalty_weight * squared_error, squared_error)

    # Calculate the mean of the weighted errors
    loss = torch.mean(weighted_error)
    return loss

In [101]:
# Assuming you have already prepared X_train, y_train, X_test, y_test

# Initialize the model
input_size = len(X_train.columns)
model = RegressionModel(input_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100  # You can change the number of epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        x, y = batch
        optimizer.zero_grad()
        y_pred = model(x)
        loss = custom_mse_loss(y_pred.squeeze(), y)  # squeeze to match y's dimensions
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


Epoch 1, Loss: 850922449483.687
Epoch 2, Loss: 685345748760.4869
Epoch 3, Loss: 534341884313.6
Epoch 4, Loss: 392737989035.4087
Epoch 5, Loss: 289649098645.1478
Epoch 6, Loss: 238943232178.08694
Epoch 7, Loss: 212796616864.27826
Epoch 8, Loss: 199176840735.16522
Epoch 9, Loss: 191024303736.2087
Epoch 10, Loss: 185316675156.5913
Epoch 11, Loss: 180537035232.83478
Epoch 12, Loss: 177040579165.49567
Epoch 13, Loss: 173188406672.69565
Epoch 14, Loss: 170331929840.4174
Epoch 15, Loss: 167604292421.0087
Epoch 16, Loss: 165323515583.44348
Epoch 17, Loss: 162797380759.3739
Epoch 18, Loss: 161377914701.91306
Epoch 19, Loss: 159664467362.50433
Epoch 20, Loss: 157226481361.25217
Epoch 21, Loss: 156680740899.6174
Epoch 22, Loss: 155656773222.4
Epoch 23, Loss: 154057496558.1913
Epoch 24, Loss: 152912463818.5739
Epoch 25, Loss: 152270572072.06958
Epoch 26, Loss: 151866474923.4087
Epoch 27, Loss: 151241661039.30435
Epoch 28, Loss: 150480690790.4
Epoch 29, Loss: 149954745530.9913
Epoch 30, Loss: 14955

In [103]:
# Evaluate the model
model.eval()
with torch.no_grad():
    mae = 0
    for batch in test_loader:
        x, y = batch
        y_pred = model(x)
        mae += torch.abs(y - y_pred).mean().item()
    mae /= len(test_data)

print(f'Mean Absolute Error on the test set: {mae}')


Mean Absolute Error on the test set: 10948.999915081522


In [110]:
mean_price = data["price"].mean()

percentage = (mae / mean_price) * 100
print(f"MAE is {percentage:.2f}% of the mean price.")

MAE is 1.98% of the mean price.
